# Joining California's Mass Exodus

### Introduction

The Great Migration out of California, between 2008 and 2019, more than 18,000 companies have left Califrornia for more tax and regulatory friendly states.Major companies that have moved their headquiarters out of California in recent years are relocating mostly to Texas, followed by Nevada, Arizona, Colorado, Oregon, and Washington. For example, big companies like Tesla and Oracle are changing their corporate headquarters to Austin, Texas. In the near future, more companies will be a part of this migration due to anti-bussiness laws and regularions passed catching up. Other reasons why California became a less favorable place to live in is because of the high taxes (13% state income tax), high housing costs, cost of living in California is about 50 percent higher than the median. Additionally, with the flexibility from remote work rise in 2020, people are willing to leave the Golden State despite a pay cut.

With the consideration of moving out of California, it is important to first explore potential relocating neighborhood, comparing similarity and dissimilarity to the currently resided neighborhood. Downtown San Jose in San Jose, California. For the simplicity of this project, Austin, TX will be chosen for comparison. By applying methods and techniques learned from this data science course, we wish to find the most optimal relocation spot to continue the journey of chasing the American dream.

#### Target audience

This project targets to give first impression of Austin, Texas to people who are considering moving from Silicon valley.

### Data Description

Importing libraries to help us looking at the data. Code is directly imported from previous exercise with changes in installing geopy and folium.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


We first collect the geographical coordinates for San Jose,CA and Austin, TX.

In [2]:
addressFrom = 'San Jose, CA'
addressTo = 'Austin, TX'

geolocator = Nominatim(user_agent='my_email@myserver.com')
locationFrom = geolocator.geocode(addressFrom)
locationTo = geolocator.geocode(addressTo)
latitudeFrom = locationFrom.latitude
longitudeFrom = locationFrom.longitude
latitudeTo = locationTo.latitude
longitudeTo = locationTo.longitude
print(f'San Jose, CA coordinates are {latitudeFrom}, {longitudeFrom}')
print(f"Austin, TX's coordinates are {latitudeTo}, {longitudeTo}")

San Jose, CA coordinates are 37.3361905, -121.890583
Austin, TX's coordinates are 30.2711286, -97.7436995


Downtown San Jose's coordinate can be hardcoded

In [3]:
dt_sj_lat = 37.3337
dt_sj_lng = -121.8907

Neighborhood data for Austin, TX can be obtained from the Housing and Planning Department of the Austin City Government (https://jovian.ai/outlink?url=https%3A%2F%2Fdata.austintexas.gov%2FBuilding-and-Development%2FNeighborhoods%2Fa7ap-j2yt%3B)
This dataset is exported as csv format and included as asset in this project folder. includes 

In [4]:
# The code was removed by Watson Studio for sharing.

,the_geom,FID,TARGET_FID,NEIGHNAME,SqMiles,Shape_Leng,Shape__Area,Shape__Length
0,MULTIPOLYGON (((-97.792307359674 30.4567073495...,3,3,ANDERSON MILL,8.669086,154458.205390,2.416802e+08,154458.205390
1,MULTIPOLYGON (((-97.670762852964 30.3085399639...,95,95,WINDSOR PARK,2.383074,40527.378654,6.643628e+07,40527.378654
2,MULTIPOLYGON (((-97.753526659646 30.2387648363...,19,19,DAWSON,0.495535,17697.924998,1.381473e+07,17697.924998
3,MULTIPOLYGON (((-97.738154269236 30.3027463827...,91,91,WEST UNIVERSITY,0.738442,24981.044476,2.058658e+07,24981.044476
4,MULTIPOLYGON (((-97.682624533084 30.2858668623...,49,49,MLK,1.545283,33470.415430,4.308001e+07,33470.415430


In [5]:
df_Austin.shape

(103, 8)

Drop irrelavent data

In [6]:
df_Austin.drop(['FID', 'TARGET_FID', 'SqMiles','Shape_Leng', 'Shape__Area', 'Shape__Length'],axis=1, inplace=True)
df_Austin.head()

,the_geom,NEIGHNAME
0,MULTIPOLYGON (((-97.792307359674 30.4567073495...,ANDERSON MILL
1,MULTIPOLYGON (((-97.670762852964 30.3085399639...,WINDSOR PARK
2,MULTIPOLYGON (((-97.753526659646 30.2387648363...,DAWSON
3,MULTIPOLYGON (((-97.738154269236 30.3027463827...,WEST UNIVERSITY
4,MULTIPOLYGON (((-97.682624533084 30.2858668623...,MLK


This dataframe contains 103 neighborhood's data. The coordinate data is still missing from this dataset. The coordinate will be obtained from geopy.


In [7]:
df_Austin['LAT'] = None
df_Austin['LNG'] = None
df_Austin.head()

,the_geom,NEIGHNAME,LAT,LNG
0,MULTIPOLYGON (((-97.792307359674 30.4567073495...,ANDERSON MILL,None,None
1,MULTIPOLYGON (((-97.670762852964 30.3085399639...,WINDSOR PARK,None,None
2,MULTIPOLYGON (((-97.753526659646 30.2387648363...,DAWSON,None,None
3,MULTIPOLYGON (((-97.738154269236 30.3027463827...,WEST UNIVERSITY,None,None
4,MULTIPOLYGON (((-97.682624533084 30.2858668623...,MLK,None,None


In [8]:
austin_gmaps = Nominatim(user_agent='TX_explore')
for neighborhood in df_Austin['NEIGHNAME']:
    neighborhood_address = neighborhood + ' Austin, TX'
    try:
        location = austin_gmaps.geocode(neighborhood_address)
        df_Austin['LAT'].loc[df_Austin['NEIGHNAME'] == neighborhood] = location.latitude
        df_Austin['LNG'].loc[df_Austin['NEIGHNAME'] == neighborhood] = location.longitude
        #print(neighborhood, location.latitude, location.longitude)
    except:
        print(neighborhood + " has wrong name")

DITTMAR--SLAUGHTER has wrong name
MANSFIELD--RIVER PLACE has wrong name
RMMA has wrong name
HAYS WARTHA has wrong name
VILLAGE AT WESTERN OAKS has wrong name
DEL VALLE has wrong name
ROBINSON RANCH has wrong name
DEL VALLE EAST has wrong name
UPPER BOGGY CREEK has wrong name
WHISPER VALLEY has wrong name
NORTH LAMAR RUNDBERG has wrong name
AVERY RANCH--LAKELINE has wrong name
BARTON CREEK MALL has wrong name
NORTH UNIVERSITY has wrong name


Some neighborhoods' name are incorrect, they will be ignored.

In [9]:
df_Austin.dropna(inplace=True)

To further clean the dataframe, first column is removed

In [10]:
df_Austin.drop(['the_geom'],axis=1,inplace=True)
df_Austin

,NEIGHNAME,LAT,LNG
0,ANDERSON MILL,30.4558,-97.8071
1,WINDSOR PARK,30.3135,-97.6911
2,DAWSON,30.2329,-97.7614
3,WEST UNIVERSITY,30.2817,-97.7388
4,MLK,30.284,-97.694
5,SOUTH RIVER CITY,30.2494,-97.7456
6,MLK-183,30.2794,-97.6739
7,OLD WEST AUSTIN,30.2968,-97.7549
8,BOULDIN CREEK,30.2557,-97.7555
9,WALNUT CREEK--PIONEER HILL,30.3789,-97.6534


In [11]:
df_Austin.shape

(89, 3)

89 neighborhoods remaining. Let's visualize where all these neighborhood are located.

In [12]:
map_Austin = folium.Map(location=[latitudeTo, longitudeTo], zoom_start=11)

for lat, lng, label in zip(df_Austin['LAT'], df_Austin['LNG'], df_Austin['NEIGHNAME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Austin)
    
map_Austin

After the geographical data are prepared, Foursquare API is used to explore venus within reasonable walking distance, 1km

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
VERSION = '20180605'
LIMIT = 100

Let's explore Downtown San jose, getting the top 100 venues that are within a radius of 1km.

Creating GET request URL

In [15]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    dt_sj_lat, 
    dt_sj_lng, 
    radius, 
    LIMIT)

Look at the GET request results

In [16]:
dt_sj_results = requests.get(url).json()
dt_sj_results

{'meta': {'code': 200, 'requestId': '600e6023520d1f65397de6e6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown San Jose',
  'headerFullLocation': 'Downtown San Jose, San Jose',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 153,
  'suggestedBounds': {'ne': {'lat': 37.34270000900001,
    'lng': -121.87940203264628},
   'sw': {'lat': 37.32469999099999, 'lng': -121.90199796735371}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a6baaf9f964a5208dcf1fe3',
       'name': 'Silicon Valley Capital Club',
       'location': {'address': '50 W San Fernando St',
        'lat': 37.3337893,
        'lng': -121.8892131,
      

Extracting catgory of the venus

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean data and put into pandas dataframe

In [18]:
dt_sj_venues = dt_sj_results['response']['groups'][0]['items']
    
dt_sj_nearby_venues = json_normalize(dt_sj_venues) # flatten JSON

# filter columns
dt_sj_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
dt_sj_nearby_venues =dt_sj_nearby_venues.loc[:, dt_sj_filtered_columns]

# filter the category for each row
dt_sj_nearby_venues['venue.categories'] = dt_sj_nearby_venues.apply(get_category_type, axis=1)

# clean columns
dt_sj_nearby_venues.columns = [col.split(".")[-1] for col in dt_sj_nearby_venues.columns]

dt_sj_nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Silicon Valley Capital Club,Restaurant,37.333789,-121.889213
1,Morton's The Steakhouse,Steakhouse,37.333048,-121.888907
2,San Pedro Square,Plaza,37.335179,-121.893044
3,Original Gravity Public House,Pub,37.335052,-121.889747
4,Umbrella Salon,Salon / Barbershop,37.335629,-121.892059


Same will be done on the first neighborhood in Austin, Anderson Mill

In [19]:
anderson_mill_lat = df_Austin['LAT'].iloc[0]
anderson_mill_lng = df_Austin['LNG'].iloc[0]
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    anderson_mill_lat, 
    anderson_mill_lng, 
    radius, 
    LIMIT)

In [20]:
anderson_mill_results = requests.get(url).json()
anderson_mill_results

{'meta': {'code': 200, 'requestId': '600e6024174e6005333dfadb'},
 'response': {'headerLocation': 'Village at Anderson Mill',
  'headerFullLocation': 'Village at Anderson Mill, Austin',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 30.46483450900001,
    'lng': -97.79667459193007},
   'sw': {'lat': 30.446834490999994, 'lng': -97.81751680806994}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e9c6074f5b9d590054b6c83',
       'name': 'Millrun Park',
       'location': {'address': '12100 Millwright Pkwy.',
        'lat': 30.4515483714773,
        'lng': -97.80297466900612,
        'labeledLatLngs': [{'label': 'display',
          'lat': 30.4515483714773,
          'lng': -97.80297466900612}],
        'distance': 6

In [21]:
anderson_mill_venues = anderson_mill_results['response']['groups'][0]['items']
    
anderson_mill_nearby_venues = json_normalize(anderson_mill_venues) # flatten JSON

# filter columns
anderson_mill_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
anderson_mill_nearby_venues =anderson_mill_nearby_venues.loc[:, anderson_mill_filtered_columns]

# filter the category for each row
anderson_mill_nearby_venues['venue.categories'] = anderson_mill_nearby_venues.apply(get_category_type, axis=1)

# clean columns
anderson_mill_nearby_venues.columns = [col.split(".")[-1] for col in anderson_mill_nearby_venues.columns]

anderson_mill_nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Millrun Park,Park,30.451548,-97.802975
1,Harper Park (in Anderson Mill),Park,30.457869,-97.811024
2,Freda's Seafood Grille,Seafood Restaurant,30.464196,-97.803776
3,Interstellar Bbq,BBQ Joint,30.461178,-97.814930
4,Punch Austin Kettlebell Gym,Gym / Fitness Center,30.447413,-97.810296


This is an example of the data used in the analysis in the next section. Downtown San Jose's venue data will be listed out by top 10 venus. >br>
Other neighborhoods in Austin will perform the same action, clusting will be added to group similar neighborhood together. 